In [65]:
import pandas as pd 
import numpy as np
import plotly.graph_objects as go

In [12]:
predictions_df = pd.read_csv('Data/predictions.csv')
predictions_lstm_df = pd.read_csv('Data/predictions_lstm.csv')

predictions_df = pd.merge(predictions_df, predictions_lstm_df, on=['date', 'Actual'], suffixes=['', '_LSTM'])
predictions_df.head()

,date,Actual,Random_Forest,Random_Forest_Tuned,SVR,ARIMA,ARIMAX,ARIMAX_Tuned,AdaBoost,AdaBoost_Tuned,train_set,LSTM_MSE,LSTM_MADL,train_set_LSTM
0,2022-07-27,0.021618,0.018125,0.017925,0.008004,-0.018234,0.014797,0.015504,0.003237,0.009848,1,0.001015,0.014847,1.0
1,2022-07-28,0.000365,0.001279,-0.001529,0.008004,-0.007970,0.007326,0.006820,0.003060,-0.002641,1,0.009065,0.015940,1.0
2,2022-07-29,0.016685,0.012667,0.012107,0.008004,-0.003152,0.009608,-0.002630,0.004568,0.001435,1,0.007192,0.016495,1.0
3,2022-08-01,-0.048606,-0.027246,-0.027928,0.008004,-0.000013,0.003712,-0.005140,-0.021365,-0.000124,1,0.002505,0.015115,1.0
4,2022-08-02,0.003959,-0.000420,0.000027,0.008004,-0.006010,-0.011645,0.000952,0.006255,0.003607,1,0.000879,0.013803,1.0


In [18]:
positions_df = predictions_df.copy()
for col in positions_df.columns[2:]:
    if col.startswith('train_set'):
        continue
    positions_df[col] = positions_df[col].apply(lambda x: 1 if x>0 else -1)
    # positions_df[col] = positions_df[col] * positions_df['Actual']
positions_df

,date,Actual,Random_Forest,Random_Forest_Tuned,SVR,ARIMA,ARIMAX,ARIMAX_Tuned,AdaBoost,AdaBoost_Tuned,train_set,LSTM_MSE,LSTM_MADL,train_set_LSTM
0,2022-07-27,0.021618,1,1,1,-1,1,1,1,1,1,1,1,1.0
1,2022-07-28,0.000365,1,-1,1,-1,1,1,1,-1,1,1,1,1.0
2,2022-07-29,0.016685,1,1,1,-1,1,-1,1,1,1,1,1,1.0
3,2022-08-01,-0.048606,-1,-1,1,-1,1,-1,-1,-1,1,1,1,1.0
4,2022-08-02,0.003959,-1,1,1,-1,-1,1,1,1,1,1,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459,2024-05-23,-0.008135,-1,-1,1,1,-1,1,-1,-1,0,-1,-1,0.0
460,2024-05-24,-0.004164,-1,-1,1,1,-1,-1,-1,-1,0,-1,1,0.0
461,2024-05-28,0.030664,1,1,1,1,-1,1,1,-1,0,-1,1,0.0
462,2024-05-29,0.001967,1,1,1,1,-1,1,1,1,0,-1,1,0.0


In [42]:
cumulative_returns = positions_df.copy()
cumulative_returns.set_index('date', inplace=True)
cumulative_returns.drop(columns=['train_set', 'train_set_LSTM'], inplace=True)

for col in cumulative_returns.columns[1:]:
    if col.startswith('train_set'):
        continue
    cumulative_returns[col] = (cumulative_returns[col] * cumulative_returns['Actual']) + 1
cumulative_returns = cumulative_returns.cumprod()
cumulative_returns

,Actual,Random_Forest,Random_Forest_Tuned,SVR,ARIMA,ARIMAX,ARIMAX_Tuned,AdaBoost,AdaBoost_Tuned,LSTM_MSE,LSTM_MADL
date,,,,,,,,,,,
2022-07-27,2.161759e-02,1.021618,1.021618,1.021618,0.978382,1.021618,1.021618,1.021618,1.021618,1.021618,1.021618
2022-07-28,7.886754e-06,1.021990,1.021245,1.021990,0.978025,1.021990,1.021990,1.021990,1.021245,1.021990,1.021990
2022-07-29,1.315897e-07,1.039042,1.038284,1.039042,0.961707,1.039042,1.004939,1.039042,1.038284,1.039042,1.039042
2022-08-01,-6.395985e-09,1.089545,1.088751,0.988539,1.008451,0.988539,1.053784,1.089545,1.088751,0.988539,0.988539
2022-08-02,-2.532108e-11,1.085232,1.093061,0.992452,1.004459,0.984625,1.057956,1.093859,1.093061,0.992452,0.992452
...,...,...,...,...,...,...,...,...,...,...,...
2024-05-23,-0.000000e+00,241.286509,249.340064,0.738446,0.843886,12.870082,22.052399,56.003234,58.173496,4.649528,0.616162
2024-05-24,0.000000e+00,242.291235,250.378325,0.735371,0.840372,12.923673,22.144227,56.236433,58.415733,4.668889,0.613596
2024-05-28,0.000000e+00,249.720845,258.055917,0.757920,0.866141,12.527382,22.823256,57.960865,56.624475,4.525723,0.632411


In [54]:
prices.columns[3:]

Index(['Random_Forest', 'Random_Forest_Tuned', 'SVR', 'ARIMA', 'ARIMAX',
       'ARIMAX_Tuned', 'AdaBoost', 'AdaBoost_Tuned', 'train_set', 'LSTM_MSE',
       'LSTM_MADL', 'train_set_LSTM'],
      dtype='object')

In [74]:
prices = pd.read_csv('Data/DCOILBRENTEU.csv')
prices.rename(columns={'DATE':'date', 'DCOILBRENTEU':'close'}, inplace=True)

prices = pd.merge(prices, positions_df, on='date')
prices.drop(columns=['Actual', 'train_set', 'train_set_LSTM'], inplace=True)

prices[prices.columns[3:]] = prices[prices.columns[3:]].shift(-1)

prices['date'] = pd.to_datetime(prices['date'])
prices.set_index('date', inplace=True)
prices

,close,Random_Forest,Random_Forest_Tuned,SVR,ARIMA,ARIMAX,ARIMAX_Tuned,AdaBoost,AdaBoost_Tuned,LSTM_MSE,LSTM_MADL
date,,,,,,,,,,,
2022-07-27,109.64,1,-1.0,1.0,-1.0,1.0,1.0,1.0,-1.0,1.0,1.0
2022-07-28,109.68,1,1.0,1.0,-1.0,1.0,-1.0,1.0,1.0,1.0,1.0
2022-07-29,111.51,1,-1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0,1.0
2022-08-01,106.09,-1,1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0
2022-08-02,106.51,-1,-1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
2024-05-23,79.25,-1,-1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0
2024-05-24,78.92,-1,1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0,1.0
2024-05-28,81.34,1,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,-1.0,1.0


In [95]:
# define the header and parameters
def backtest(dataframe, model, start_date=None,end_date=None):

#---------------Date Selector---------------#    

        # selecting the startdate and enddate
        # if start and enddate is not specify, then select all range
        if start_date==None:
            start_date=dataframe.index[1]
        if end_date==None:
            end_date=dataframe.index[-1]
        # slicing the range if start/end date is specified
        data=dataframe.loc[start_date:end_date,:]
        data=data[['close', model]]
        data.rename(columns={model:'signal'}, inplace=True)

#---------------Signal Generator---------------#    
        
        # data['20high']=data['open'].rolling(20*24).max()
        # data['10low']=data['open'].rolling(10*24).min()
        
        # data=data.dropna()

        # data['buysignal']=data['close']-data['20high']
        # data['sellsignal']=data['close']-data['10low']

        # def buysig(series):
        #     if series>0:
        #         return 1
        #     if series<0:
        #         return 0

        # def sellsig(series):
        #     if series>0:
        #         return 0
        #     if series<0:
        #         return -1

        # # apply the transformation function to dataframe
        # data.loc[:,'buysignal']=data.loc[:,'buysignal'].apply(lambda x: buysig(x))
        # data.loc[:,'sellsignal']=data.loc[:,'sellsignal'].apply(lambda x: sellsig(x))

        # data['signal']=data['buysignal']+data['sellsignal']

#---------------Account Setup---------------#    
        
        # setup counting system
        holding=0        ## how many bitcoin is currently holding
        holdingcost=0    ## prime cost for getting the bitcoin we are holding
        outflow=0        ## tracking on cash outflow (cumulative)
        inflow=0         ## tracking on cash inflow (cumulative)
        order={}         ## a dict should contain buy/sell date and its price

#---------------Trading Loop---------------#

        # iterating over the dataframe by iterrows to execute the order
        for date,row in data.iterrows():

            # if signal is openpos(signal=1), and there's no holding bitcoin(holding=0) then
            if row['signal']==1 and holding==0:

                # add 1 bitcoin into holding
                holding=holding+1
                # track on all-time cash outflow
                outflow-=row['close']
                # track on the single time buyin cost
                holdingcost+=row['close']
                # recortd the cost and buytime into dict
                order[date]=-row['close']
                # print out the on-time buyin action
                print('Buy:{buy_create}$ @ {dt}'.format(buy_create=order[date],dt=date))

            # else if signal is less than 0,and holding any of bitcoin then sell out all
            elif row['signal']< 0 and holding>0:

                # track the all-time cash inflow (before selling)
                inflow+= row['close']*holding
                # record the sell action into dict             
                order[date]=+row['close']*holding
                # reset the holding number (since we have sold all)
                holding=0
                # reset the holding cost (since we don't hold anything now)
                holdingcost=0
                # print out the on-time sell out action
                print('Sell:{sell_create}$ @ {dt}'.format(sell_create=round(order[date],2),dt=date)) ## round

            # record bitcoin holding qty in dataframe 
            data.loc[date,'holding']=holding
            # record currently portfolio value in dataframe
            data.loc[date,'portvalue']=data.loc[date,'holding']*row['close']
            # record currently holding cost in dataframe
            data.loc[date,'holdingcost']=holdingcost

            # record profit in each timestamp
            data.loc[date,'profit']=inflow+outflow+data.loc[date,'portvalue']

#---------------Print Out Result---------------#

        # calculation of the ROI
        data['roi']=(data['portvalue']-data['holdingcost'])/data['holdingcost']
        data['roi']=data['roi'].fillna(0)

        # print a seperator
        print('------------------------Summary------------------------')
        # print the total profit made
        print('Profit(Incl Bitcoin Holding):{profit:} $'.format(profit=round((outflow+inflow+data.loc[data.index[-1],'portvalue'])))) ## round 
        # print the total profit made
        print('Maximum Holding Cost(USDT):{profit:} $'.format(profit=(round(data['holdingcost'].max())))) ## round
        # print the total time used
        print('Duration:{timeframe:}'.format(timeframe=(data.index[-1]-data.index[0]))) 
        # print the maximum drawdown rate
        print('Max Drawdown:{drawdown:} %'.format(drawdown=round(100*data['roi'].min())))

#---------------Plot Nicely with Plotly---------------#        

        # create empty list for saving buy_date and sell_date
        buy_date=[]
        sell_date=[]

        # loop over the order dictionary and append buy & sell date
        for key,value in order.items():
            if value>0:
                sell_date.append(key)
            if value<0:
                buy_date.append(key)

        # extract the buy price and sell price
        sp=data.loc[sell_date,'close']
        bp=data.loc[buy_date,'close']
        
        # setup plotly object figure
        fig=go.Figure()

        # adding trace of close price and profit
        fig.add_trace(go.Scatter(x=data.index,y=data['close'],name='Close',line_color='SlateBlue'))

        fig.add_trace(go.Scatter(x=data.index,y=data['profit'],name='Profit',line_color='LightPink'))

        # adding marker of Buyin and sell out 
        fig.add_trace(go.Scatter(x=buy_date,y=np.abs(bp),
                                name='Buy',mode='markers',
                                marker=dict(size=10,symbol=5),
                                marker_color='red',
                                text='BUY'))

        fig.add_trace(go.Scatter(x=sell_date,y=np.abs(sp),
                                name='Sell',mode='markers',
                                marker=dict(size=10,symbol=6),
                                marker_color='lime',
                                text='SELL'))

        # styling the plot and put text on it
        fig.update_layout(title_text=f'Backtest:{model}',
                        xaxis_rangeslider_visible=True,
                        template='gridon')

        # show the plot
        fig.show()

        # return the data into dataframe
        return data

In [77]:
prices.columns

Index(['close', 'Random_Forest', 'Random_Forest_Tuned', 'SVR', 'ARIMA',
       'ARIMAX', 'ARIMAX_Tuned', 'AdaBoost', 'AdaBoost_Tuned', 'LSTM_MSE',
       'LSTM_MADL'],
      dtype='object')

In [96]:
# Random_Forest_Tuned, ARIMAX, ARIMAX_Tuned, AdaBoost, LSTM_MSE
backtest(prices, 'LSTM_MSE')

Buy:-109.68$ @ 2022-07-28 00:00:00
Sell:106.51$ @ 2022-08-02 00:00:00
Buy:-100.31$ @ 2022-08-05 00:00:00
Sell:103.46$ @ 2022-08-08 00:00:00
Buy:-95.36$ @ 2022-08-16 00:00:00
Sell:96.35$ @ 2022-08-18 00:00:00
Buy:-96.45$ @ 2022-08-19 00:00:00
Sell:99.49$ @ 2022-08-23 00:00:00
Buy:-86.83$ @ 2022-09-07 00:00:00
Sell:91.68$ @ 2022-09-09 00:00:00
Buy:-92.83$ @ 2022-09-14 00:00:00
Sell:89.28$ @ 2022-09-15 00:00:00
Buy:-89.43$ @ 2022-09-16 00:00:00
Sell:89.86$ @ 2022-09-21 00:00:00
Buy:-84.29$ @ 2022-09-23 00:00:00
Sell:89.55$ @ 2022-09-28 00:00:00
Buy:-88.9$ @ 2022-09-30 00:00:00
Sell:93.74$ @ 2022-10-04 00:00:00
Buy:-95.65$ @ 2022-10-06 00:00:00
Sell:97.13$ @ 2022-10-10 00:00:00
Buy:-89.46$ @ 2022-10-18 00:00:00
Sell:91.52$ @ 2022-10-20 00:00:00
Buy:-91.57$ @ 2022-10-24 00:00:00
Sell:91.76$ @ 2022-10-25 00:00:00
Buy:-92.93$ @ 2022-10-26 00:00:00
Sell:94.64$ @ 2022-10-28 00:00:00
Buy:-94.25$ @ 2022-11-10 00:00:00
Sell:96.37$ @ 2022-11-11 00:00:00
Buy:-94.3$ @ 2022-11-15 00:00:00
Sell:92.61$ 

,close,signal,holding,portvalue,holdingcost,profit,roi
date,,,,,,,
2022-07-28,109.68,1.0,1.0,109.68,109.68,0.00,0.000000
2022-07-29,111.51,1.0,1.0,111.51,109.68,1.83,0.016685
2022-08-01,106.09,1.0,1.0,106.09,109.68,-3.59,-0.032732
2022-08-02,106.51,-1.0,0.0,0.00,0.00,-3.17,0.000000
2022-08-03,101.82,-1.0,0.0,0.00,0.00,-3.17,0.000000
...,...,...,...,...,...,...,...
2024-05-23,79.25,-1.0,0.0,0.00,0.00,55.61,0.000000
2024-05-24,78.92,-1.0,0.0,0.00,0.00,55.61,0.000000
2024-05-28,81.34,-1.0,0.0,0.00,0.00,55.61,0.000000
